**Reading and processing data**

In [1]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization
from sklearn.model_selection import train_test_split
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

In [2]:
test = 'test.csv'
train = 'train.csv'

data = pd.read_csv(train)
target = data["Target"]
data = data.drop(columns=["Id", "Target", "idhogar"])
data['dependency'] = data['dependency'].map({'yes': 1, 'no': 0})
data['edjefe'] = data['edjefe'].map({'yes': 1, 'no': 0})
data['edjefa'] = data['edjefa'].map({'yes': 1, 'no': 0})


test = pd.read_csv(test)
ID = test["Id"]
test = test.drop(columns=["Id", "idhogar"])
test['dependency'] = test['dependency'].map({'yes': 1, 'no': 0})
test['edjefe'] = test['edjefe'].map({'yes': 1, 'no': 0})
test['edjefa'] = test['edjefa'].map({'yes': 1, 'no': 0})

In [3]:
X_train, X_test, y_train, y_test = train_test_split(data, target, test_size=0.3, random_state=0)
X_train = X_train.fillna(X_train.mean())
X_test = X_test.fillna(X_test.mean())

test = test.fillna(test.mean())

In [4]:
from sklearn.preprocessing import StandardScaler

scaler = StandardScaler()
X_scaled = scaler.fit_transform(X_train)
X_test_scaled = scaler.fit_transform(X_test)


test_scaled = scaler.fit_transform(test)

In [5]:
from imblearn.over_sampling import RandomOverSampler
from imblearn.combine import SMOTETomek
from imblearn.under_sampling import RandomUnderSampler

def oversample(X, y):
    ros = RandomOverSampler(random_state=0)
    return ros.fit_resample(X, y)

def undersample(X, y):
    ros = RandomUnderSampler(random_state=0)
    return ros.fit_resample(X, y)

def smotsample(X, y):
    ros = SMOTETomek(sampling_strategy='auto')
    return ros.fit_resample(X, y)

In [6]:
undersampled_X, undersampled_y = undersample(X_scaled, y_train)
oversampled_X, oversampled_y = oversample(X_scaled, y_train)
smotsampled_X, smotsampled_y = smotsample(X_scaled, y_train)

from keras.utils import to_categorical

undersampled_y_onehot = to_categorical(undersampled_y-1)
oversampled_y_onehot = to_categorical(oversampled_y-1)
smotsampled_y_onehot = to_categorical(smotsampled_y-1)

y_train_onehot = to_categorical(y_train-1)

**Training models**

In [7]:
model = Sequential()
model.add(Dense(256, activation='relu', input_dim=X_scaled.shape[1]))
model.add(Dense(128, activation='relu'))
model.add(Dense(64, activation='relu'))
model.add(Dense(32, activation='relu'))
model.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model.add(Dense(4, activation='softmax'))

model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [8]:
import tensorflow as tf
tf.config.run_functions_eagerly(True)
model.fit(X_scaled, y_train_onehot, epochs=100, batch_size=1000, validation_split=0.2)

Epoch 1/100


/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


6/6 [==============================] - 2s 215ms/step - loss: 1.6036 - accuracy: 0.2833 - val_loss: 1.0980 - val_accuracy: 0.6457
Epoch 2/100
6/6 [==============================] - 1s 237ms/step - loss: 1.2675 - accuracy: 0.4608 - val_loss: 1.0141 - val_accuracy: 0.6734
Epoch 3/100
6/6 [==============================] - 1s 155ms/step - loss: 1.1625 - accuracy: 0.5423 - val_loss: 0.9611 - val_accuracy: 0.6771
Epoch 4/100
6/6 [==============================] - 1s 124ms/step - loss: 1.0690 - accuracy: 0.6079 - val_loss: 0.9268 - val_accuracy: 0.6764
Epoch 5/100
6/6 [==============================] - 1s 131ms/step - loss: 0.9899 - accuracy: 0.6573 - val_loss: 0.9039 - val_accuracy: 0.6839
Epoch 6/100
6/6 [==============================] - 1s 95ms/step - loss: 0.9140 - accuracy: 0.7023 - val_loss: 0.8885 - val_accuracy: 0.6898
Epoch 7/100
6/6 [==============================] - 1s 87ms/step - loss: 0.8435 - accuracy: 0.7382 - val_loss: 0.8784 - val_accuracy: 0.6936
Epoch 8/100
6/6 [==========

In [9]:
model_under = Sequential()
model_under.add(Dense(256, activation='relu', input_dim=undersampled_X.shape[1]))
model_under.add(Dense(128, activation='relu'))
model_under.add(Dense(64, activation='relu'))
model_under.add(Dense(32, activation='relu'))
model_under.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model_under.add(Dense(4, activation='softmax'))

model_under.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [10]:
model_under.fit(undersampled_X, undersampled_y_onehot, epochs=100, batch_size=1000, validation_split=0.2)

Epoch 1/100
2/2 [==============================] - 0s 284ms/step - loss: 1.4287 - accuracy: 0.2164 - val_loss: 1.4399 - val_accuracy: 0.0310
Epoch 2/100
2/2 [==============================] - 0s 186ms/step - loss: 1.3593 - accuracy: 0.3132 - val_loss: 1.5185 - val_accuracy: 0.0000e+00
Epoch 3/100
2/2 [==============================] - 0s 155ms/step - loss: 1.3277 - accuracy: 0.3365 - val_loss: 1.6200 - val_accuracy: 0.0000e+00
Epoch 4/100
2/2 [==============================] - 0s 105ms/step - loss: 1.2876 - accuracy: 0.3897 - val_loss: 1.7606 - val_accuracy: 0.0000e+00
Epoch 5/100
2/2 [==============================] - 0s 105ms/step - loss: 1.2402 - accuracy: 0.4375 - val_loss: 1.9339 - val_accuracy: 0.0000e+00
Epoch 6/100
2/2 [==============================] - 0s 105ms/step - loss: 1.1935 - accuracy: 0.4710 - val_loss: 2.1442 - val_accuracy: 0.0000e+00
Epoch 7/100
2/2 [==============================] - 0s 114ms/step - loss: 1.1497 - accuracy: 0.4860 - val_loss: 2.3525 - val_accuracy: 

In [11]:
model_over = Sequential()
model_over.add(Dense(256, activation='relu', input_dim=oversampled_X.shape[1]))
model_over.add(Dense(128, activation='relu'))
model_over.add(Dense(64, activation='relu'))
model_over.add(Dense(32, activation='relu'))
model_over.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model_over.add(Dense(4, activation='softmax'))

model_over.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [12]:
model_over.fit(oversampled_X, oversampled_y_onehot, epochs=100, batch_size=1000, validation_split=0.2)

Epoch 1/100
14/14 [==============================] - 1s 83ms/step - loss: 1.2634 - accuracy: 0.3970 - val_loss: 2.5937 - val_accuracy: 0.0000e+00
Epoch 2/100
14/14 [==============================] - 1s 72ms/step - loss: 1.0044 - accuracy: 0.5903 - val_loss: 2.8109 - val_accuracy: 0.0000e+00
Epoch 3/100
14/14 [==============================] - 1s 70ms/step - loss: 0.8212 - accuracy: 0.7031 - val_loss: 2.5580 - val_accuracy: 0.0000e+00
Epoch 4/100
14/14 [==============================] - 1s 91ms/step - loss: 0.6489 - accuracy: 0.7774 - val_loss: 2.3699 - val_accuracy: 0.0000e+00
Epoch 5/100
14/14 [==============================] - 1s 107ms/step - loss: 0.5190 - accuracy: 0.8165 - val_loss: 2.2138 - val_accuracy: 0.0029
Epoch 6/100
14/14 [==============================] - 1s 104ms/step - loss: 0.4304 - accuracy: 0.8470 - val_loss: 2.0792 - val_accuracy: 0.0165
Epoch 7/100
14/14 [==============================] - 1s 76ms/step - loss: 0.3597 - accuracy: 0.8701 - val_loss: 1.8108 - val_accur

In [13]:
model_smot = Sequential()
model_smot.add(Dense(256, activation='relu', input_dim=smotsampled_X.shape[1]))
model_smot.add(Dense(128, activation='relu'))
model_smot.add(Dense(64, activation='relu'))
model_smot.add(Dense(32, activation='relu'))
model_smot.add(Dense(16, activation='relu'))
model.add(BatchNormalization())
model_smot.add(Dense(4, activation='softmax'))

model_smot.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

In [14]:
model_smot.fit(smotsampled_X, smotsampled_y_onehot, epochs=100, batch_size=1000, validation_split=0.2)

Epoch 1/100
14/14 [==============================] - 1s 88ms/step - loss: 1.1465 - accuracy: 0.5061 - val_loss: 2.8538 - val_accuracy: 0.0000e+00
Epoch 2/100
14/14 [==============================] - 1s 84ms/step - loss: 0.8588 - accuracy: 0.6854 - val_loss: 2.5220 - val_accuracy: 0.0000e+00
Epoch 3/100
14/14 [==============================] - 1s 78ms/step - loss: 0.6950 - accuracy: 0.7610 - val_loss: 2.3073 - val_accuracy: 0.0000e+00
Epoch 4/100
14/14 [==============================] - 1s 80ms/step - loss: 0.5665 - accuracy: 0.8094 - val_loss: 2.3352 - val_accuracy: 0.0000e+00
Epoch 5/100
14/14 [==============================] - 1s 82ms/step - loss: 0.4687 - accuracy: 0.8385 - val_loss: 2.0993 - val_accuracy: 0.0127
Epoch 6/100
14/14 [==============================] - 1s 80ms/step - loss: 0.3850 - accuracy: 0.8676 - val_loss: 1.8857 - val_accuracy: 0.0419
Epoch 7/100
14/14 [==============================] - 2s 119ms/step - loss: 0.3259 - accuracy: 0.8862 - val_loss: 1.7255 - val_accura

In [15]:
predictions = model.predict(X_test_scaled)
predictions2 = model_under.predict(X_test_scaled)
predictions3 = model_over.predict(X_test_scaled)
predictions4 = model_smot.predict(X_test_scaled)

90/90 [==============================] - 1s 6ms/step


In [16]:
from sklearn.metrics import accuracy_score

predictions = np.argmax(predictions, axis=1) + 1
predictions2 = np.argmax(predictions2, axis=1) + 1
predictions3 = np.argmax(predictions3, axis=1) + 1
predictions4 = np.argmax(predictions4, axis=1) + 1

accuracy = accuracy_score(y_test, predictions)
accuracy2 = accuracy_score(y_test, predictions2)
accuracy3 = accuracy_score(y_test, predictions3)
accuracy4 = accuracy_score(y_test, predictions4)
print("Accuracy of all data:", accuracy)
print("Accuracy of undersampling:", accuracy2)
print("Accuracy of oversampling:", accuracy3)
print("Accuracy of smot sampling:", accuracy4)

Accuracy of all data: 0.75278940027894
Accuracy of undersampling: 0.43654114365411434
Accuracy of oversampling: 0.8165969316596932
Accuracy of smot sampling: 0.7932357043235704


**Preparing Kaggle submission**

In [22]:
preds = model_over.predict(test_scaled)
preds = np.argmax(preds, axis=1) + 1

  5/746 [..............................] - ETA: 9s 

/usr/local/lib/python3.10/dist-packages/tensorflow/python/data/ops/structured_function.py:258: UserWarning: Even though the `tf.config.experimental_run_functions_eagerly` option is set, this option does not apply to tf.data functions. To force eager execution of tf.data functions, please use `tf.data.experimental.enable_debug_mode()`.
  warnings.warn(


746/746 [==============================] - 9s 12ms/step


In [38]:
df = pd.DataFrame({'ID': ID, 'Target': preds})
df.to_csv('submission.csv', index=False)